<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.2: Combinational Logic
**Prev: [Your First Chisel Module](2.1_first_module.ipynb)**<br>
**Next: [Control Flow](2.3_control_flow.ipynb)**

## Motivation
In this section you will see how to use Chisel components to implement combinational logic.
We will demonstrate how three of the basic Chisel types: `UInt `- unsigned integer; `SInt` - signed integer, and `Bool` - true or false may be connected and operated upon.UInt,SInt,Bool,**这三个类型是chisel独有的,而并非从scala继承**
Notice how all Chisel variables are declared as Scala `val`s.
Never use a Scala `var` for a hardware construct, since the construct itself may never change once defined; only its value may change when running the hardware.
Wires may be used for parameterized types.

## Setup

# 这一小节的关键是

int 会被看做是整数的值,而Uint会被看做是硬件节点.事实上,scala根本没有1.U这个类型!!!!而且scala本身是一个强类型的语言,所以1.U和1相加会报错.一个是硬件,一个是scala中的变量,一定不能弄错

声明一个硬件元件的时候,用num.UInt,创建chisel bool的方法则是true.B.scala中,boolean才是他的是非类型

在类里面定义的变量,就一定不能够用:=,而是用=.

Mux和cat是chisel独用的方法

In [2]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(os.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling /home/ding/MyChiselBootCamp/Main.scCompiling /home/ding/MyChiselBootCamp/Main.sc #2

path: String = "/home/ding/MyChiselBootCamp/source/load-ivy.sc"

In [3]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

---
# Common Operators
Now that you understand how `Module`s are constructed, let's make some hardware! Take a look at the empty module below.

In [4]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
}

defined class MyModule

We've called our class `MyModule`, and it extends `Module`. This means it gets mapped to a hardware module in Verilog. Our `MyModule` module has one input and one output. The input is a 4-bit unsigned integer (`UInt`), and so is the output. 

<span style="color:blue">**Example: Scala and Chisel Operators Look the Same**</span><br>scala和chisel的操作符看上去并没有区别

Let's look at different operations we can perform on data.来看看不同的可以在数据上面运行的操作符

In [66]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val two  = 1 + 1
  println(two)
  val utwo = 1.U + 1.U//scala把所有东西都看做对象,所以这里也是一个转换,把1转换成1.U
  println("在这里打印出来类型名称和指针\n" + utwo)
  
  io.out := io.in
  printf(s"在这里打印出来仿真的时候能访问的值%d",io.in)
  //printf("%d",two) 是错误的,因为只有在仿真的时候才能用到printf,访问不了

}
println(getVerilog(new MyModule))

Elaborating design...
2
在这里打印出来类型名称和指针
UInt<1>(OpResult in MyModule)
Done elaborating.
module MyModule(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd65.sc 12:10]
  always @(posedge clock) begin
    `ifndef SYNTHESIS
    `ifdef PRINTF_COND
      if (`PRINTF_COND) begin
    `endif
        if (~reset) begin
          $fwrite(32'h80000002,"?????????????????%d",io_in); // @[cmd65.sc 13:9]
        end
    `ifdef PRINTF_COND
      end
    `endif
    `endif // SYNTHESIS
  end
endmodule



defined class MyModule

We create two `val`s. The first adds two Scala `Int`s, so `println` prints out the integer 2. The second `val` adds two *Chisel* `UInt`s together, so `println` sees this as a hardware node and prints out the type name and pointer (`chisel3.core.UInt@d`). Note that `1.U` is a type cast from a Scala `Int` (1) to a Chisel `UInt` literal.

We need to drive the output to something, so we just connect it to the input for now, as with the passthrough module in the previous tutorial.

<span style="color:blue">**Example: Incompatible Operation**</span><br>
What happens if we add a Chisel `1.U` to the literal `1`? These types are incompatible, as the former is a hardware wire of value 1, while the latter is a Scala value of 1. So Chisel will give a type mismatch error.

In [11]:
class MyModuleTwo extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val twotwo = 1.U + 1//前者是线,后者是值,不兼容
  println(twotwo)
  
  io.out := io.in
}
println(getVerilog(new MyModuleTwo))

cmd11.sc:7: type mismatch;
 found   : Int(1)
 required: chisel3.UInt
  val twotwo = 1.U + 1
                     ^Compilation Failed

: 

It's important to remember the distinction between types when performing operations. Scala is a strongly typed language, so any type casting must be explicit.

<span style="color:blue">**Example: More Chisel Operators**</span><br>
Other common operations are subtraction and multiplication. These are handled on unsigned integers as expected. Let's  see these in action. We show the Verilog, though there's some underlying Chisel features that obfuscate the simple code we would expect.

In [18]:
class MyOperators extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_add = Output(UInt(4.W))
    val out_sub = Output(UInt(4.W))
    val out_mul = Output(UInt(4.W))
  })

  io.out_add := 1.U + 4.U//直接输出了4‘h5，没有用到wire
  io.out_sub := 2.U - 1.U//用wire new了一个减法的结果，然后用到了cat
  io.out_mul := 4.U * 2.U//用wire完成操作
}
println(getVerilog(new MyOperators))

Elaborating design...
Done elaborating.
module MyOperators(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out_add,
  output [3:0] io_out_sub,
  output [3:0] io_out_mul
);
  wire [1:0] _T_3 = 2'h2 - 2'h1; // @[cmd17.sc 10:21]
  wire [4:0] _T_4 = 3'h4 * 2'h2; // @[cmd17.sc 11:21]
  assign io_out_add = 4'h5; // @[cmd17.sc 9:21]
  assign io_out_sub = {{2'd0}, _T_3}; // @[cmd17.sc 10:21]
  assign io_out_mul = _T_4[3:0]; // @[cmd17.sc 11:14]
endmodule



defined class MyOperators

And here's a sample tester for the above operations. Instead of using an anonymous tester class like in the previous tutorial, we'll create an explicit tester class. This is just an alternative way of writing a tester.

In [9]:
test(new MyOperators) {c =>
  c.io.out_add.expect(5.U)
  c.io.out_sub.expect(1.U)
  c.io.out_mul.expect(8.U)
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
test MyOperators Success: 0 tests passed in 2 cycles in 0.022341 seconds 89.52 Hz
SUCCESS!!


<span style="color:blue">**Example: Mux and Concatenation**</span><br>
In addition to addition, subtraction, and multplication, Chisel has mux and concatenation operators. These are shown below. The `Mux` operates like a traditional ternary operator, with the order (select, value if true, value if false). Note that `true.B` and `false.B` are the preferred ways to create Chisel Bool literals. The `Cat` ordering is MSB then LSB (where B refers to bit or bits), and only takes two arguments. Concatenating more than two values requires multiple `Cat` calls or advanced Chisel and Scala features covered in later sections.

chisel允许了多路复用器mux和连接器cat。mux类似三元符号，（boolean，iftrue，iffalse）

true.B和false.B是创建bool的方法,cat的参数顺序是高位 + 地位.链接多个值需要多个cat

In [25]:
class MyOperatorsTwo extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_mux = Output(UInt(4.W))
    val out_cat = Output(UInt(4.W))
  })

  val s = false.B
  val nas = true.B
  io.out_mux := Mux(s, 3.U, 0.U) // should return 3.U, since s is true
  io.out_cat := Cat(2.U, 2.U)    // concatenates 2 (b10) with 1 (b1) to give 5 (101)
    //如果在这里输入过多,会固定到f,而不继续做扩展
}

println(getVerilog(new MyOperatorsTwo))

class MyOwnOperator extends Module {
  val io = IO(new Bundle {
    val inA      = Input(UInt(4.W))
    val inB      = Input(UInt(4.W))
      
    val inBool  = Input(Bool())
    val out_a = Output(UInt(4.W))
    val out_b = Output(UInt(4.W))
  })

  
  io.out_a := Mux(io.inBool, io.inA, io.inB) // should return 3.U, since s is true
  io.out_b := Cat(io.inA, io.inB)    // concatenates 2 (b10) with 1 (b1) to give 5 (101)
}

println(getVerilog(new MyOwnOperator))


Elaborating design...
Done elaborating.
module MyOperatorsTwo(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out_mux,
  output [3:0] io_out_cat
);
  assign io_out_mux = 4'h0; // @[cmd24.sc 10:20]
  assign io_out_cat = 4'ha; // @[Cat.scala 30:58]
endmodule

Elaborating design...
Done elaborating.
module MyOwnOperator(
  input        clock,
  input        reset,
  input  [3:0] io_inA,
  input  [3:0] io_inB,
  input        io_inBool,
  output [3:0] io_out_a,
  output [3:0] io_out_b
);
  wire [7:0] _T_1 = {io_inA,io_inB}; // @[Cat.scala 30:58]
  assign io_out_a = io_inBool ? io_inA : io_inB; // @[cmd24.sc 28:18]
  assign io_out_b = _T_1[3:0]; // @[cmd24.sc 29:12]
endmodule



defined class MyOperatorsTwo
defined class MyOwnOperator

Notice how the Verilog contains constants instead of actual mux or concatenation logic. This is because FIRRTL transformations have simplified the circuit, eliminating obvious logic. 

上面的例子里面,只包括张亮,因为firrtl完成了简化,优化逻辑.下面例子是我自己写的

For a more complete list of Chisel operators, see the [Chisel cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf). For the most complete list of operators and their implementation details, look through the [Chisel API](https://chisel-lang.org/api/latest/).

---
# Exercises
To complete these exercises, you may need to look through the [Chisel cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf).

<span style="color:red">**Exercise: MAC**</span><br>
Create a Chisel module that implements the multiply accumulate function, `(A*B)+C`, and passes the testbench.

In [26]:
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  io.out := (io.in_a * io.in_b ) + io.in_c 
}

println(getVerilog(new MAC))

test(new MAC) { c =>
  val cycles = 100
  import scala.util.Random
  for (i <- 0 until cycles) {
    val in_a = Random.nextInt(16)
    val in_b = Random.nextInt(16)
    val in_c = Random.nextInt(16)
    c.io.in_a.poke(in_a.U)
    c.io.in_b.poke(in_b.U)
    c.io.in_c.poke(in_c.U)
    c.io.out.expect((in_a * in_b + in_c).U)
  }
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
module MAC(
  input        clock,
  input        reset,
  input  [3:0] io_in_a,
  input  [3:0] io_in_b,
  input  [3:0] io_in_c,
  output [7:0] io_out
);
  wire [7:0] _T = io_in_a * io_in_b; // @[cmd25.sc 9:22]
  wire [7:0] _GEN_0 = {{4'd0}, io_in_c}; // @[cmd25.sc 9:34]
  assign io_out = _T + _GEN_0; // @[cmd25.sc 9:34]
endmodule

Elaborating design...
Done elaborating.
test MAC Success: 0 tests passed in 2 cycles in 0.040783 seconds 49.04 Hz
SUCCESS!!


defined class MAC

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  io.out := (io.in_a * io.in_b) + io.in_c
}
</pre></article></div></section></div>

<span style="color:red">**Exercise: Arbiter**</span><br>
The following circuit arbitrates data coming from a FIFO into two parallel processing units. The FIFO and processing elements (PEs) communicate with ready-valid interfaces. Construct the arbiter to send data to whichever PE is ready to receive data, prioritizing PE0 if both are ready to receive data. Remember that the arbiter should tell the FIFO that it's ready to receive data when at least one of the PEs can receive data. Also, wait for a PE to assert that it's ready before asserting that the data are valid. You will likely need binary operators to complete this exercise.

<img src="images/arbiter.png" width="687" height="177">

In [58]:
class Arbiter extends Module {
  val io = IO(new Bundle {
    // FIFO
    val fifo_valid = Input(Bool())
    val fifo_ready = Output(Bool())
    val fifo_data  = Input(UInt(16.W))
    
    // PE0
    val pe0_valid  = Output(Bool())
    val pe0_ready  = Input(Bool())
    val pe0_data   = Output(UInt(16.W))
    
    // PE1
    val pe1_valid  = Output(Bool())
    val pe1_ready  = Input(Bool())
    val pe1_data   = Output(UInt(16.W))
  })

    
    //io.fifo_ready := io.pe0_ready || io.pe1_ready
    
    //io.pe0_valid := io.fifo_valid && io.pe0_ready
    //io.pe1_valid := io.fifo_valid && io.pe1_ready && !io.pe0_valid
    
    //io.pe0_data := io.fifo_data
    //io.pe1_data := io.fifo_data
    
    //above is sulution 1,aka the solution given by bootcamp itself    
    io.fifo_ready := false.B
    io.pe0_valid := false.B
    io.pe1_valid := false.B
    when(io.pe0_ready || io.pe1_ready){io.fifo_ready := true.B}
    when(io.fifo_valid && io.pe0_ready){io.pe0_valid := true.B}
    when(io.fifo_valid && io.pe1_ready && !io.pe0_ready){io.pe1_valid := true.B}
    io.pe0_data := io.fifo_data
    io.pe1_data := io.fifo_data
    
    //但是这样写有一个问题:要初始化为false的地方太多了,而且有when,这是不鼓励的
    }

test(new Arbiter) { c =>
  import scala.util.Random
  val data = Random.nextInt(65536)
  c.io.fifo_data.poke(data.U)
  
  for (i <- 0 until 8) {
    c.io.fifo_valid.poke((((i >> 0) % 2) != 0).B)
    c.io.pe0_ready.poke((((i >> 1) % 2) != 0).B)
    c.io.pe1_ready.poke((((i >> 2) % 2) != 0).B)

    c.io.fifo_ready.expect((i > 1).B)
    c.io.pe0_valid.expect((i == 3 || i == 7).B)
    c.io.pe1_valid.expect((i == 5).B)
    
    if (i == 3 || i ==7) {
      c.io.pe0_data.expect((data).U)
    } else if (i == 5) {
      c.io.pe1_data.expect((data).U)
    }
  }
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
test Arbiter Success: 0 tests passed in 2 cycles in 0.003563 seconds 561.29 Hz
SUCCESS!!


defined class Arbiter

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  io.fifo_ready := io.pe0_ready || io.pe1_ready
  io.pe0_valid := io.fifo_valid && io.pe0_ready
  io.pe1_valid := io.fifo_valid && io.pe1_ready && !io.pe0_ready
  io.pe0_data := io.fifo_data
  io.pe1_data := io.fifo_data
</pre></article></div></section></div>

<span style="color:red">**Exercise: Parameterized Adder (Optional)**</span><br>
This optional exercise exposes you to one of the most powerful features of Chisel, it's parameterization capabilities. To demonstrate this, we'll construct a parameterized adder that can either saturate the output when overflow occurs, or truncate the results (i.e. wrap around).

saturate 处理  truncate 截断

First, look at the `Module` below. The parameter we pass into it is called `saturate` and has type *Scala* `Boolean`.**这是scala的boolean而不是chisel的bool!!!!** This is not a Chisel `Bool`. So, we're not creating a single hardware adder that can either saturate or truncate, but rather we're creating a *generator* that produces either a saturating hardware adder *or* a truncating hardware adder. The decision is made at compile time.

Next, notice the inputs and outputs are all 4-bit `UInt`s. Chisel has built-in width inferencing, and if you look at the [cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf), you'll see that the bitwidth of a normal summation is equal to the maximum bitwidth of the two inputs. This means that

```scala
val sum = io.in_a + io.in_b
```

will make `sum` a 4-bit wire, and the value will be the truncated result for 4-bit inputs. To check if the summation should saturate, you need to place the result in a 5-bit wire. This can be done with the `+&` summation, as seen on the cheatsheet.

```scala
val sum = io.in_a +& io.in_b
```

Finally, note that connecting a 4-bit `UInt` wire to a 5-bit `UInt` wire will truncate the MSB by default. You can use this to easily truncate the 5-bit sum for the non-saturating adder.

In [69]:
class ParameterizedAdder(saturate: Boolean) extends Module {//boolean是scala的类型!
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val out  = Output(UInt(4.W))
  })

    val sum = io.in_a +& io.in_b
    if(saturate)
        io.out := Mux(sum > 15.U, 15.U,sum)//这里要u因为这是一个wire,而不是一个scala中的变量
    else io.out := sum
}

println(getVerilog(new ParameterizedAdder(true)))//true.B是chisel的,而不是scala的

for (saturate <- Seq(true, false)) {
  test(new ParameterizedAdder(saturate)) { c =>
    // 100 random tests
    val cycles = 100
    import scala.util.Random
    import scala.math.min
    for (i <- 0 until cycles) {
      val in_a = Random.nextInt(16)
      val in_b = Random.nextInt(16)
      c.io.in_a.poke(in_a.U)
      c.io.in_b.poke(in_b.U)
      if (saturate) {
        c.io.out.expect(min(in_a + in_b, 15).U)
      } else {
        c.io.out.expect(((in_a + in_b) % 16).U)
      }
    }
    
    // ensure we test saturation vs. truncation
    c.io.in_a.poke(15.U)
    c.io.in_b.poke(15.U)
    if (saturate) {
      c.io.out.expect(15.U)
    } else {
      c.io.out.expect(14.U)
    }
  }
}

println("SUCCESS!!")

Elaborating design...
Done elaborating.
module ParameterizedAdder(
  input        clock,
  input        reset,
  input  [3:0] io_in_a,
  input  [3:0] io_in_b,
  output [3:0] io_out
);
  wire [4:0] sum = io_in_a + io_in_b; // @[cmd68.sc 8:23]
  wire [4:0] _T_1 = sum > 5'hf ? 5'hf : sum; // @[cmd68.sc 10:22]
  assign io_out = _T_1[3:0]; // @[cmd68.sc 10:16]
endmodule

Elaborating design...
Done elaborating.
test ParameterizedAdder Success: 0 tests passed in 2 cycles in 0.004884 seconds 409.53 Hz
Elaborating design...
Done elaborating.
test ParameterizedAdder Success: 0 tests passed in 2 cycles in 0.003880 seconds 515.42 Hz
SUCCESS!!


defined class ParameterizedAdder

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val sum = io.in_a +& io.in_b
  if (saturate) {
    io.out := Mux(sum > 15.U, 15.U, sum)
  } else {
    io.out := sum
  }
</pre></article></div></section></div>